In [1]:
import re
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
#examine html for list of recipe names under "main dishes" category on allrecipes
pagecnt = 1
url = 'https://www.allrecipes.com/recipes/80/main-dish/?page='+pagecnt
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
soup


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>Main Dish Recipes - Allrecipes.com</title>
<script async="true" src="https://secureimages.allrecipes.com/assets/deployables/v-1.135.0.4208/karma.bundled.js"></script>
<script async="true" src="https://secureimages.allrecipes.com/assets/deployables/v-1.135.0.4208/foresee.bundled.js"></script>
<!--Make our website baseUrl available to the client-side code-->
<script type="text/javascript">
        var AR = AR || {};
        AR.baseWebsiteUrl = 'https://www.allrecipes.com';
    </script>
<script type="text/javascript">
        //Remove Ref_Hub from session after first recipe visited
        var hubId = window.sessionStorage["Ref_Hub_Id"];
        var count = window.sessionStorage["Ref_Hub_Recipe_Count"];
        if (hubId && count) {
            if (count > 0) {
                window.sessionStorage.removeItem("Ref_Hub_Id");
                window.sessionStorage.removeItem("Ref_Hub_Recipe_Count");
            }
        }
    </script>
<m

In [20]:
links = soup.findAll('a',attrs={'class':'fixed-recipe-card__title-link'})
links = [link['href'] for link in links]

In [21]:
names = soup.findAll('span',attrs={'class':'fixed-recipe-card__title-link'})
names = [name.contents[0] for name in names]

In [22]:
ratings = soup.findAll('span',attrs={'class':'stars stars-4-5'})
ratings = [rating['data-ratingstars'] for rating in ratings]

In [23]:
numreviews = soup.findAll('span',attrs={'class':'fixed-recipe-card__reviews'})
numreviews = [review.contents[0]['number'] for review in numreviews]

In [98]:
recipedata = zip(names,ratings,numreviews,links)
recipedata = [x for x in recipedata if int(x[2])>0]
pd.DataFrame(recipedata,columns=['recipenames','ratings','numreviews','links']).set_index('recipenames').head()

,ratings,numreviews,links
recipenames,,,
Grilled Chicken with Peach Sauce,4.41860485076904,63,https://www.allrecipes.com/recipe/223175/grill...
Chicago Deep-Dish Pizza,4.7380952835083,2,https://www.allrecipes.com/recipe/263611/chica...
Chimichurri Baked Chicken,4.44067811965942,7,https://www.allrecipes.com/recipe/261124/chimi...
Philly Cheese Steak-Stuffed Bell Peppers,4.71875,6,https://www.allrecipes.com/recipe/263799/phill...
Easy Tuna Noodle Casserole,4.53526210784912,5,https://www.allrecipes.com/recipe/263813/easy-...


In [112]:
#extra recipe names, ratings, numreviews, links by looping thru pages of main disheds on allrecipes
pagecnts = range(1,100)
allrecipedata = []
for cnt in pagecnts:
    try:
        url = 'https://www.allrecipes.com/recipes/80/main-dish/?page='+str(cnt)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')

        links = soup.findAll('a',attrs={'class':'fixed-recipe-card__title-link'})
        links = [link['href'] for link in links]
        names = soup.findAll('span',attrs={'class':'fixed-recipe-card__title-link'})
        names = [name.contents[0] for name in names]
        ratings = soup.findAll('span',attrs={'class':'stars stars-4-5'})
        ratings = [rating['data-ratingstars'] for rating in ratings]
        numreviews = soup.findAll('span',attrs={'class':'fixed-recipe-card__reviews'})
        numreviews = [review.contents[0]['number'] for review in numreviews]

        recipedata = zip(names,ratings,numreviews,links)
        recipedata = [x for x in recipedata if int(x[2])>0]
        allrecipedata.extend(recipedata)
    except:
        print('error with page:',cnt)
    
    if cnt % 20 == 0:
        print('finished page:',cnt)
    time.sleep(1)

allrecipes_db = pd.DataFrame(allrecipedata,columns=['recipenames','ratings','numreviews','links']).set_index('recipenames')
allrecipes_db.to_csv('allrecipes_db3.csv',index=False) #should've used index=True
allrecipes_db.head()

finished page: 20
finished page: 40
finished page: 60
finished page: 80


In [31]:
#examining the html for nutrition facts page on allrecipes.com
url = recipedata[0][3]
url = url+'fullrecipenutrition'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
soup


<div class="recipe-nutrition">
<h1 class="nutrition_h1">Nutritional Information</h1>
<h2>Grilled Chicken with Peach Sauce</h2>
<div class="nutrition-top light-underline">

        Servings Per Recipe: 8<br/>
<span class="bold">Amount Per Serving</span><br/>
<span class="bold">Calories:</span> 406<br/>
</div>
<div class="nutrition-body-label">% Daily Value *</div>
<div class="nutrition-body">
<div class="nutrition-row">
<span class="nutrient-name">Total Fat: <span class="nutrient-value">7.9g</span></span>
<span class="daily-value">12 %</span>
</div>
<div class="nutrition-row">
<span class="nutrient-name indent">Saturated Fat: <span class="nutrient-value">2.0g</span></span>
</div>
<div class="nutrition-row">
<span class="nutrient-name">Cholesterol: <span class="nutrient-value">67mg</span></span>
<span class="daily-value">22 %</span>
</div>
<div class="nutrition-row">
<span class="nutrient-name">Sodium: <span class="nutrient-value">333mg</span></span>
<span class="daily-value">13 %</span

In [55]:
calories = [x for x in soup.find('div',attrs={'class':'nutrition-top light-underline'}).stripped_strings]
calories = int(calories[-1])
calories

406

In [73]:
nutrientnames = soup.find_all('span',attrs={'class':'nutrient-name'})
nutrientnames = [x.contents[0].replace(': ','') for x in nutrientnames]

In [75]:
nutrientvals = soup.find_all('span',attrs={'class':'nutrient-value'})
nutrientvals = [x.contents[0].replace(': ','') for x in nutrientvals]

In [80]:
nutrinfo = dict(x for x in zip(nutrientnames,nutrientvals))
nutrinfo

{'Total Fat': '7.9g',
 'Saturated Fat': '2.0g',
 'Cholesterol': '67mg',
 'Sodium': '333mg',
 'Potassium': '231mg',
 'Total Carbohydrates': '59.3g',
 'Dietary Fiber': '0.1g',
 'Protein': '24.8g',
 'Sugars': '54g',
 'Vitamin A': '18IU',
 'Vitamin C': '21mg',
 'Calcium': '20mg',
 'Iron': '1mg',
 'Thiamin': '0mg',
 'Niacin': '16mg',
 'Vitamin B6': '0mg',
 'Magnesium': '26mg',
 'Folate': '4mcg'}

In [82]:
pd.DataFrame.from_dict({recipedata[0][0]:nutrinfo},orient='index')

,Total Fat,Saturated Fat,Cholesterol,Sodium,Potassium,Total Carbohydrates,Dietary Fiber,Protein,Sugars,Vitamin A,Vitamin C,Calcium,Iron,Thiamin,Niacin,Vitamin B6,Magnesium,Folate
Grilled Chicken with Peach Sauce,7.9g,2.0g,67mg,333mg,231mg,59.3g,0.1g,24.8g,54g,18IU,21mg,20mg,1mg,0mg,16mg,0mg,26mg,4mcg
